In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
vocab_size=10000
max_length=200

In [3]:
data=pd.read_csv("./bbc_news_mixed.csv")

In [4]:
data.head()

,text,label
0,Cairn shares slump on oil setback\n\nShares in...,business
1,Egypt to sell off state-owned bank\n\nThe Egyp...,business
2,Cairn shares up on new oil find\n\nShares in C...,business
3,Low-cost airlines hit Eurotunnel\n\nChannel Tu...,business
4,"Parmalat to return to stockmarket\n\nParmalat,...",business


In [5]:
data['label'].value_counts()

label
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [6]:
data['text'].iloc[0]

'Cairn shares slump on oil setback\n\nShares in Cairn Energy, a UK oil firm, have closed down 18% after a disappointing drilling update and a warning over possible tax demands.\n\nThe company said tests had shown no significant finds in one of its Indian oil fields, but was upbeat about the potential of other areas. It also said the Indian government had told it to pay a production tax, for which Cairn argues it is not liable. Cairn\'s shares have jumped by almost 400% this year. Investors had piled into Cairn after the company announced significant oil finds in India this year. Chief executive Bill Gammell said on Friday he was "disappointed" with exploration in the so-called N-C extension area in Rajasthan. Investors had held high hopes of major oil finds in this area. But Cairn said estimates had been revised in what was a "significant downgrade of the initial expectation".\n\nCairn also said that the government believed the company was liable to pay taxes under its production-shari

In [7]:
token=Tokenizer(num_words=vocab_size)

In [8]:
token.fit_on_texts(data['text'])

In [9]:
seq=token.texts_to_sequences(data['text'])

In [10]:
pad_seq=pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')

In [11]:
model=Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 1042565 (3.98 MB)
Trainable params: 1042565 (3.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [22]:
# encoder=LabelEncoder()
encoder=OneHotEncoder()

In [24]:
y=encoder.fit_transform(data[['label']])

In [ ]:
categorical_labels=to_categorical(y)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
pad_seq=pad_seq.astype('float32')

In [28]:
xtrain, xtest, ytrain, ytest=train_test_split(pad_seq, categorical_labels, train_size=0.8)

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(xtrain, ytrain, epochs=10)

Epoch 1/10


56/56 [==============================] - 12s 151ms/step - loss: 0.2598 - accuracy: 0.9365
Epoch 2/10
56/56 [==============================] - 9s 155ms/step - loss: 0.1386 - accuracy: 0.9663
Epoch 3/10
56/56 [==============================] - 10s 175ms/step - loss: 0.3211 - accuracy: 0.9388
Epoch 4/10
56/56 [==============================] - 10s 175ms/step - loss: 0.4689 - accuracy: 0.8455
Epoch 5/10
56/56 [==============================] - 9s 158ms/step - loss: 0.2657 - accuracy: 0.9483
Epoch 6/10
56/56 [==============================] - 9s 157ms/step - loss: 0.1404 - accuracy: 0.9753
Epoch 7/10
56/56 [==============================] - 9s 158ms/step - loss: 0.0471 - accuracy: 0.9933
Epoch 8/10
56/56 [==============================] - 9s 152ms/step - loss: 0.0207 - accuracy: 0.9989
Epoch 9/10
56/56 [==============================] - 9s 163ms/step - loss: 0.0258 - accuracy: 0.9955
Epoch 10/10
56/56 [==============================] - 9s 154ms/step - loss: 0.0422 - accuracy: 0.9893


In [31]:
model.save('bbcmodel.h5')

C:\Users\jubim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
import joblib

In [34]:
joblib.dump(encoder,'encoder.pkl')
joblib.dump(token,'tokenizer.pkl')

['tokenizer.pkl']

In [35]:
config={}

In [36]:
config['vocab_size'] = vocab_size

In [37]:
config['pad_length']=max_length

In [40]:
file=open('config.txt', 'w')

In [41]:
file.write(str(config))

40